# Day 22 - More pathfinding, adding weights

At first this appears to be a matrix problem, but the modulus over 3 prime numbers rules that out quickly. Part two reveals that this is actually a pathfinding problem, with part 1 just setting you up with a map generator.

In [1]:
from itertools import product
from typing import List, Tuple

M: int = 20183
X0: int = 16807
Y0: int = 48271
    
Pos = Tuple[int, int]

class ModeMaze:
    def __init__(self, depth: int, target: Pos) -> None:
        self.depth = depth
        self.target = target
        # y rows, x columns
        self._erosion_levels: List[List[int]] = []
        self._width, self._height = 0, 0

    def _gen_levels(self, pos: Pos):
        # make sure there is enough erosion level info up to the given position
        width, height = pos[0] + 1, pos[1] + 1
        cw, ch = self._width, self._height
        if cw >= width and ch >= height:
            return
        
        depth = self.depth
        maze = self._erosion_levels
        
        # add more rows
        for y in range(ch, height):
            # "or 1" to generate the first column when empty
            above = maze[y - 1] if y else [(x * X0 + depth) % M for x in range(cw or 1)]
            row = [(y * Y0 + self.depth) % M]
            for p in above[1:]:
                row.append((p * row[-1] + self.depth) % M)
            maze.append(row)
        
        # add more columns
        for x in range(cw or 1, width):
            rows = iter(maze)
            next(rows).append((x * X0 + depth) % M)
            for p, row in zip(maze, rows):
                row.append((p[-1] * row[-1] + depth) % M)
        
        if self.target[1] >= cw and self.target[0] >= ch:
            maze[self.target[1]][self.target[0]] = self.depth
            
        self._width, self._height = width, height
            
    @property
    def risk_level(self):
        self._gen_levels(self.target)
        tx, ty = self.target
        return sum(sum(i % 3 for i in row[:tx + 1]) for row in self._erosion_levels[:ty + 1])
    
    def __str__(self):
        lines = [['.=|'[i % 3] for i in row] for row in self._erosion_levels]
        lines[0][0] = 'M'
        lines[self.target[1]][self.target[0]] = 'T'
        return '\n'.join([''.join(l) for l in lines])    

In [2]:
test = ModeMaze(510, (10, 10))
assert test.risk_level == 114
test._gen_levels((15, 15))
assert str(test) == """\
M=.|=.|.|=.|=|=.
.|=|=|||..|.=...
.==|....||=..|==
=.|....|.==.|==.
=|..==...=.|==..
=||.=.=||=|=..|=
|.=.===|||..=..|
|..==||=.|==|===
.=..===..=|.|||.
.======|||=|=.|=
.===|=|===T===||
=|||...|==..|=.|
=.=|=.=..=.||==|
||=|=...|==.=|==
|=.=||===.|||===
||.|==.|.|.||=||"""

In [3]:
import aocd
import re

data = aocd.get_data(day=22, year=2018)
depth, tx, ty = map(int, re.findall(r'\d+', data))
maze = ModeMaze(depth, (tx, ty))

In [4]:
print('Part 1:', maze.risk_level)

Part 1: 8090
